<a href="https://colab.research.google.com/github/JeanMusenga/PhD-Thesis_2024_Musenga/blob/main/Luhn_TextSummarizationWith.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Important resources:
https://chatgpt.com/share/0fde4435-eb1d-4ab1-9149-78c0899251e0

Importing necessary libraries

In [1]:
import re
import nltk
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

In [14]:
dataset=pd.read_excel('DataSampePilot.xlsx')

In [3]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Preprocessing function

In [15]:
# Preprocessing function
def preprocess_text(text):
    sentences = re.split(r'(?<=[.!?]) +', text)  # Simple sentence tokenizer
    processed_sentences = []

    for sentence in sentences:
        words = re.findall(r'\b\w+\b', sentence.lower())  # Tokenization
        words = [word for word in words if word not in ENGLISH_STOP_WORDS]
        processed_sentences.append(words)

    return sentences, processed_sentences

# Calculate word frequency function

In [16]:
# Calculate word frequency function
def calculate_word_frequency(processed_sentences):
    word_freq = {}
    for sentence in processed_sentences:
        for word in sentence:
            if word not in word_freq:
                word_freq[word] = 1
            else:
                word_freq[word] += 1
    return word_freq

# Score sentences function

In [17]:
# Score sentences function
def score_sentences(original_sentences, processed_sentences, word_freq):
    sentence_scores = np.zeros(len(original_sentences))
    for i, sentence in enumerate(processed_sentences):
        for word in sentence:
            if word in word_freq:
                sentence_scores[i] += word_freq[word]
    return sentence_scores

# Generate summary function

In [18]:
# Generate summary function
def generate_summary(original_sentences, scores, num_sentences=3):
    ranked_sentence_indices = np.argsort(scores)[::-1]
    summary = []
    for i in range(min(num_sentences, len(original_sentences))):
        summary.append(original_sentences[ranked_sentence_indices[i]])
    return " ".join(summary)

In [19]:
# Preprocess the Question_body and Answer_body columns
dataset['processed_question'] = dataset['Question_body'].apply(preprocess_text)
dataset['processed_answer'] = dataset['Answer_body'].apply(preprocess_text)

In [21]:
# Calculate word frequencies
dataset['question_word_freq'] = dataset['processed_question'].apply(lambda x: calculate_word_frequency(x[1]))
dataset['answer_word_freq'] = dataset['processed_answer'].apply(lambda x: calculate_word_frequency(x[1]))


In [22]:
# Score sentences
dataset['question_scores'] = dataset.apply(lambda x: score_sentences(x['processed_question'][0], x['processed_question'][1], x['question_word_freq']), axis=1)
dataset['answer_scores'] = dataset.apply(lambda x: score_sentences(x['processed_answer'][0], x['processed_answer'][1], x['answer_word_freq']), axis=1)


In [23]:
# Generate summaries for both questions and answers
dataset['question_summary'] = dataset.apply(lambda x: generate_summary(x['processed_question'][0], x['question_scores']), axis=1)
dataset['answer_summary'] = dataset.apply(lambda x: generate_summary(x['processed_answer'][0], x['answer_scores']), axis=1)


In [25]:
# Display the summaries
summaries = dataset[['question_summary', 'answer_summary']]
summaries.head()

,question_summary,answer_summary
0,"When all is done, itâ€™ll show a response.\nSo...",Single container can usually handle hundreds o...
1,If I use Spring MVC for designing the client s...,The term &quot;<em>Microservices</em>&quot; is...
2,On which layer should I implement the logic fo...,"Your repositories might return the same model,..."
3,But that's trouble for another time.\nMy wider...,"Use a URL like this, replacing xxxx with your ..."
4,"I am trying to learn AWS services, and now it ...",It would also facilitate SSL (in addition to t...


In [26]:
# Save the summarized data to a new Excel file
file_path = '/content/TextRank_SummarizedData.xlsx'

dataset.to_excel(file_path, index=False, engine='openpyxl')

In [30]:
# Verify that the file has been saved correctly
saved_data = pd.read_excel(file_path)
print(saved_data[['question_summary', 'answer_summary']].head())

                                    question_summary  \
0  When all is done, itâ€™ll show a response.\nSo...   
1  If I use Spring MVC for designing the client s...   
2  On which layer should I implement the logic fo...   
3  But that's trouble for another time.\nMy wider...   
4  I am trying to learn AWS services, and now it ...   

                                      answer_summary  
0  Single container can usually handle hundreds o...  
1  The term &quot;<em>Microservices</em>&quot; is...  
2  Your repositories might return the same model,...  
3  Use a URL like this, replacing xxxx with your ...  
4  It would also facilitate SSL (in addition to t...  
